Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape([-1, image_size, image_size]).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000, 10)
Validation set (10000, 28, 28) (10000, 10)
Test set (10000, 28, 28) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable( initial )

In [6]:
def bias_variable(shape):
    return tf.Variable( tf.zeros(shape) )    

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [8]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
        strides=[1, 2, 2, 1], padding='SAME')

In [12]:
graph = tf.Graph()
with graph.as_default():
    
    keep_prob = tf.placeholder( tf.float32 )
    tf_train_x = tf.placeholder( tf.float32, shape=[None, image_size, image_size] )
    tf_train_y = tf.placeholder( tf.float32, shape=[None, num_labels] )
    tf_train_image = tf.reshape( tf_train_x, [-1, image_size, image_size, 1])
    
    w1 = weight_variable([5, 5, 1, 32])
    b1 = bias_variable([32])
    h1_relu = tf.nn.relu( conv2d(tf_train_image, w1) + b1 )
    h1 = tf.nn.dropout( max_pool_2x2(h1_relu), keep_prob )
    
    w2 = weight_variable([5, 5, 32, 64])
    b2 = bias_variable([64])
    h2_relu = tf.nn.relu( conv2d(h1, w2) + b2 )
    h2 = tf.nn.dropout( max_pool_2x2(h2_relu), keep_prob )
    
    flat_size = int(image_size*image_size*4)
    h2_flat = tf.reshape(h2, [-1, flat_size])
    
    w1_fc = weight_variable([flat_size, 1024])
    b1_fc = bias_variable([1024])
    h1_fc = tf.nn.dropout( tf.nn.relu(tf.matmul(h2_flat, w1_fc) + b1_fc),
                           keep_prob )
    
    w2_fc = weight_variable([1024, num_labels])
    b2_fc = bias_variable([num_labels])
    logits = tf.matmul(h1_fc, w2_fc) + b2_fc
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_y, logits=logits))
    
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
    predictions = tf.equal( tf.argmax(logits, 1), tf.argmax(tf_train_y, 1) )
    accuracy = tf.reduce_mean( tf.cast(predictions, tf.float32) )

In [13]:
train_size = train_dataset.shape[0]
num_steps = 1001
batch_size = 100
num_batch = int(train_size/batch_size)
offset = 0

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print ("Initialized" )
    for step in range(num_steps):
        batch_x = train_dataset[offset:offset+batch_size, :]
        batch_y = train_labels[offset:offset+batch_size, :]
        offset += batch_size;
        if ( offset >= train_size ) : offset = 0
        if step % 100 == 0 :
            train_accuracy = accuracy.eval( feed_dict = {
                tf_train_x : batch_x, tf_train_y : batch_y, keep_prob  : 1.0 })
            print ('step %d: train_accuracy %g '% (step, train_accuracy) )
        train_step.run( feed_dict = {
            tf_train_x : batch_x, tf_train_y : batch_y, keep_prob : 0.5 })
    print ('valid accuracy %g '% accuracy.eval( feed_dict = {
        tf_train_x : valid_dataset, tf_train_y : valid_labels, keep_prob : 1.0 }))
    print ('test accuracy %g '% accuracy.eval( feed_dict = {
        tf_train_x : test_dataset, tf_train_y : test_labels, keep_prob : 1.0 }))    

Initialized
step 0: train_accuracy 0.09 
step 100: train_accuracy 0.59 
step 200: train_accuracy 0.7 
step 300: train_accuracy 0.75 
step 400: train_accuracy 0.66 
step 500: train_accuracy 0.74 
step 600: train_accuracy 0.76 
step 700: train_accuracy 0.76 
step 800: train_accuracy 0.78 
step 900: train_accuracy 0.76 
step 1000: train_accuracy 0.79 
valid accuracy 0.7875 
test accuracy 0.8466 


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
